## Capstone Project

In [65]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
! pip install geocoder
import geocoder
from pandas import ExcelWriter
from pandas import ExcelFile

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import io

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: i

### Scraping and Wrangling data

In [44]:
url = 'https://github.com/pmsanchezb/Capstone_project_IBM/blob/master/localidades_bogota.csv'
html = urlopen(url)
bs = BeautifulSoup(html,"html.parser")

### Extract table

In [45]:
table_body=bs.find('tbody')
rows = table_body.find_all('tr')
pc_bogota = []
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    pc_bogota.append(cols)

Convert to dataframe

In [46]:
df_bogota = pd.DataFrame(pc_bogota)
df_bogota.columns = ['N','Locality', 'UPZ','Neighborhood']
df_bogota = df_bogota.drop('N', 1)

In [47]:
df_bogota.head()

,Locality,UPZ,Neighborhood
0,Usaquén,Paseo de los libertadores,Canaima
1,Usaquén,Paseo de los libertadores,La floresta de la sabana
2,Usaquén,Paseo de los libertadores,Torca
3,Usaquén,Verbenal,Balmoral norte
4,Usaquén,Verbenal,Buenavista


### Get latitude and longitude

In [48]:

# initialize your variable to None
df_coord = pd.DataFrame(index=df_bogota.index,columns=['Latitude','Longitude'])
j = 0
for neighs in df_bogota['Neighborhood']: 
    lat_lng_coords = None
    i = 0
    # loop until you get the coordinates
    while(lat_lng_coords is None)&(i<300):
        g = geocoder.arcgis('{}, Bogota, Colombia'.format(neighs))
        lat_lng_coords = g.latlng
        i=i+1
        if lat_lng_coords != None:
            latitude = lat_lng_coords[0]
            longitude = lat_lng_coords[1]
            df_coord.iloc[j] = lat_lng_coords
            print(lat_lng_coords)
    j = j + 1

[4.7740400000000704, -74.04108999999994]
[4.697340000000054, -74.07561999999996]
[4.808210000000031, -74.03010999999998]
[4.703899974464835, -74.04576998167808]
[4.766430000000071, -74.02589999999998]
[4.763280000000066, -74.02437999999995]
[4.766080000000045, -74.01597999999996]
[4.76144000000005, -74.03677999999996]
[4.762550000000033, -74.04010999999997]
[4.738140025535299, -74.07411998167812]
[4.761380000000031, -74.02193999999997]
[4.6128100000000245, -74.06670999999994]
[4.603530000000035, -74.07234999999997]
[4.763150033325538, -74.04384999610485]
[4.714299948929583, -74.1264299961048]
[4.610250000000065, -74.07285999999993]
[4.552190000000053, -74.15882999999997]
[4.763100000000065, -74.02334999999994]
[4.756880000000024, -74.02828999999997]
[4.758860000000027, -74.03740999999997]
[4.783330000000035, -74.03332999999998]
[4.62587000000002, -74.17981999999995]
[4.625020000000063, -74.19787999999994]
[4.627240000000029, -74.07864999999998]
[4.753030000000024, -74.02531999999997]
[

Join df_coord to df_bogota

In [49]:
df_bogota_coord = pd.concat([df_bogota, df_coord], axis=1)
df_bogota_coord.head()

,Locality,UPZ,Neighborhood,Latitude,Longitude
0,Usaquén,Paseo de los libertadores,Canaima,4.77404,-74.0411
1,Usaquén,Paseo de los libertadores,La floresta de la sabana,4.69734,-74.0756
2,Usaquén,Paseo de los libertadores,Torca,4.80821,-74.0301
3,Usaquén,Verbenal,Balmoral norte,4.7039,-74.0458
4,Usaquén,Verbenal,Buenavista,4.76643,-74.0259


In [50]:

address = 'Bogota'

geolocator = Nominatim(user_agent="bogota_explorer")
location = geolocator.geocode(address)
latitude_bogota = location.latitude
longitude_bogota = location.longitude
print('The geograpical coordinate of Bogota are {}, {}.'.format(latitude_bogota, longitude_bogota))

The geograpical coordinate of Bogota are 4.5980772, -74.0761028.


In [51]:
# Create map of Toronto using latitude and longitude values
map_bogota = folium.Map(location=[latitude_bogota, longitude_bogota], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_bogota_coord['Latitude'], df_bogota_coord['Longitude'], df_bogota_coord['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bogota)  
    
map_bogota

In [3]:
# The code was removed by Watson Studio for sharing.

### Explore neighborhoods in Bogotá

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
bogota_venues = getNearbyVenues(names=df_bogota_coord['Neighborhood'],
                                   latitudes=df_bogota_coord['Latitude'],
                                   longitudes=df_bogota_coord['Longitude']
                                  )

Canaima
La floresta de la sabana
Torca
Balmoral norte
Buenavista
Chaparral
El codito
El refugio de San Antonio
El verbenal
Horizontes
La estrellita
La frontera
La llanurita
Marantá
Maturín
Medellín
Mirador del norte
Nuevo horizonte
San antonio norte
Santandersito
Tibabita
Viña del mar
Bosque de San Antonio
El pite
El redil
La cita
La granja norte
La uribe
Los naranjos
San juan bosco
Urbanización los laureles
Ainsuca
Altablanca
Barrancas
California
Cerro norte
Danubio
La perla oriental
Milán
Pradera norte
San cristóbal norte
San cristóbal norte parte alta
San cristóbal norte parte baja
Santa teresa
Torcoroma
Villa nydia
Villa oliva
El toberín
Babilonia
Darandelos
Estrella del norte
Guanoa
Jardín norte
La liberia
La pradera norte
Las orquídeas
Santa mónica
Villa magdala
Villas de aranjuez
Villas del mediterraneo
Zaragoza
Acacias
Antigua
Belmira
Bosque de pinos
Caobos salazar
Capri
Cedritos
Cedro bolívar
Cedro golf
Cedro madeira
Cedro narvaez
Cedro salazar
El contador
El rincón de las mar

In [57]:
print(bogota_venues.shape)
bogota_venues.head()

(7494, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Canaima,4.77404,-74.04109,Pavo Munich,4.776968,-74.041793,Sandwich Place
1,Canaima,4.77404,-74.04109,Tugo Calle 197,4.775796,-74.042030,Furniture / Home Store
2,Canaima,4.77404,-74.04109,Club el Rancho,4.774056,-74.045122,Golf Course
3,Canaima,4.77404,-74.04109,Flores De La 200,4.776016,-74.041630,Flower Shop
4,Canaima,4.77404,-74.04109,Hamburguesas El Corral,4.771193,-74.042496,Burger Joint


In [66]:
# one hot encoding
bogota_onehot = pd.get_dummies(bogota_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bogota_onehot['Neighborhood'] = bogota_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bogota_onehot.columns[-1]] + list(bogota_onehot.columns[:-1])
bogota_onehot = bogota_onehot[fixed_columns]
bogota_onehot.head()

,Neighborhood,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,University,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Canaima,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
bogota_grouped = bogota_onehot.groupby('Neighborhood').mean().reset_index()
bogota_grouped

,Neighborhood,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,University,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Acacias,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
1,Acevedo tejada,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
2,Ainsuca,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.014706,0.000000,0.0
3,Alfonso López norte,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.032258,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
4,Altablanca,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
5,Antigua,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.057143,0.000000,0.0
6,Antiguo country,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.014706,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
7,Armenia,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
8,Atahualpa,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0
9,Babilonia,0.040000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0


In [68]:
num_top_venues = 5

for hood in bogota_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bogota_grouped[bogota_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acacias----
                       venue  freq
0        Sporting Goods Shop   0.5
1  Latin American Restaurant   0.5
2            Paintball Field   0.0
3                Outlet Mall   0.0
4      Outdoors & Recreation   0.0


----Acevedo tejada----
                       venue  freq
0                 Restaurant  0.22
1                      Diner  0.11
2  Latin American Restaurant  0.06
3          Mobile Phone Shop  0.06
4       Caribbean Restaurant  0.06


----Ainsuca----
                venue  freq
0          Restaurant  0.13
1                Café  0.06
2  Italian Restaurant  0.06
3        Burger Joint  0.06
4                 Bar  0.04


----Alfonso López norte----
                       venue  freq
0  South American Restaurant  0.10
1                        Bar  0.06
2                 Restaurant  0.06
3       Gym / Fitness Center  0.06
4       Fast Food Restaurant  0.03


----Altablanca----
                     venue  freq
0               Donut Shop  0.12
1           Farmers Market

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [118]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bogota_grouped['Neighborhood']

for ind in np.arange(bogota_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bogota_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acacias,Latin American Restaurant,Sporting Goods Shop,Event Space,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Yoga Studio
1,Acevedo tejada,Restaurant,Diner,Latin American Restaurant,Department Store,Seafood Restaurant,Café,Juice Bar,Caribbean Restaurant,Steakhouse,Chinese Restaurant
2,Ainsuca,Restaurant,Italian Restaurant,Café,Burger Joint,Bar,Coffee Shop,South American Restaurant,Tapas Restaurant,BBQ Joint,Asian Restaurant
3,Alfonso López norte,South American Restaurant,Bar,Gym / Fitness Center,Restaurant,Mexican Restaurant,Café,BBQ Joint,Food Court,Steakhouse,Salon / Barbershop
4,Altablanca,Farmers Market,Health & Beauty Service,Skating Rink,Restaurant,Food & Drink Shop,Gym / Fitness Center,Fast Food Restaurant,Donut Shop,Exhibit,Fabric Shop


### Clustering Data

k = 10 clusters. I hope one cluster to have a high concentration of hotels.

In [119]:
# set number of clusters
kclusters = 12

bogota_grouped_clustering = bogota_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bogota_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([ 3,  1,  1, 11, 11, 11, 11, 11,  3,  3], dtype=int32)

In [120]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bogota_merged = df_bogota_coord

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bogota_merged = bogota_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bogota_merged.head() 

,Locality,UPZ,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Usaquén,Paseo de los libertadores,Canaima,4.77404,-74.0411,11.0,Furniture / Home Store,Department Store,Burger Joint,Grocery Store,Tennis Stadium,Bus Station,BBQ Joint,Golf Course,Sandwich Place,Flower Shop
1,Usaquén,Paseo de los libertadores,La floresta de la sabana,4.69734,-74.0756,3.0,South American Restaurant,Pizza Place,Deli / Bodega,Bakery,Latin American Restaurant,Farmers Market,Event Space,Food Truck,Supermarket,Steakhouse
2,Usaquén,Paseo de los libertadores,Torca,4.80821,-74.0301,1.0,Athletics & Sports,Soccer Field,Restaurant,Latin American Restaurant,Farm,Falafel Restaurant,Factory,Fabric Shop,Exhibit,Dog Run
3,Usaquén,Verbenal,Balmoral norte,4.7039,-74.0458,11.0,Clothing Store,Seafood Restaurant,Italian Restaurant,Café,Bakery,Hotel,Coffee Shop,Spa,Sushi Restaurant,Donut Shop
4,Usaquén,Verbenal,Buenavista,4.76643,-74.0259,11.0,Fast Food Restaurant,Bus Station,Dog Run,Clothing Store,Lighthouse,Brewery,Yoga Studio,Event Service,Eastern European Restaurant,Electronics Store


### Analize clusters and identify the best locality/neighborhood for opnening a new hotel

#### Examining cluster 1

In [121]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 0, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Usaquén,Capri,0.0,Shopping Mall,Gym,Park,Food & Drink Shop,Mexican Restaurant,Restaurant,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant
74,Usaquén,El rincón de las margaritas,0.0,Construction & Landscaping,Shopping Mall,Grocery Store,Restaurant,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
76,Usaquén,Las margaritas,0.0,Construction & Landscaping,Shopping Mall,Grocery Store,Restaurant,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
93,Usaquén,La glorieta,0.0,Shopping Mall,Food & Drink Shop,Park,Yoga Studio,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service
224,Fontibón,Fontibón centro,0.0,Shopping Mall,Gym,Park,Food & Drink Shop,Mexican Restaurant,Restaurant,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant
246,Fontibón,Florencia,0.0,Seafood Restaurant,Shopping Mall,Park,Restaurant,Eastern European Restaurant,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
249,Fontibón,La estación,0.0,Shopping Mall,Gym,Seafood Restaurant,Hotel,Restaurant,Mexican Restaurant,Yoga Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant
280,Fontibón,Mallorca,0.0,Shopping Mall,Supermarket,Pharmacy,Café,Fast Food Restaurant,Fabric Shop,Exhibit,Factory,Event Space,Donut Shop
286,Fontibón,La rosita,0.0,Shopping Mall,Food,Bar,Taco Place,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service


#### Examining cluster 2

In [122]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 1, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Usaquén,Torca,1.0,Athletics & Sports,Soccer Field,Restaurant,Latin American Restaurant,Farm,Falafel Restaurant,Factory,Fabric Shop,Exhibit,Dog Run
11,Usaquén,La frontera,1.0,Restaurant,Café,Italian Restaurant,Coffee Shop,Peruvian Restaurant,Sandwich Place,Park,Tapas Restaurant,Bar,Mexican Restaurant
12,Usaquén,La llanurita,1.0,Café,Restaurant,BBQ Joint,South American Restaurant,Seafood Restaurant,Burger Joint,Coffee Shop,Italian Restaurant,Nightclub,Diner
21,Usaquén,Viña del mar,1.0,Restaurant,Department Store,Gym,Yoga Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service
23,Usaquén,El pite,1.0,Restaurant,Latin American Restaurant,Dessert Shop,Convenience Store,Theater,Tea Room,Café,BBQ Joint,Student Center,Bakery
31,Usaquén,Ainsuca,1.0,Restaurant,Italian Restaurant,Café,Burger Joint,Bar,Coffee Shop,South American Restaurant,Tapas Restaurant,BBQ Joint,Asian Restaurant
43,Usaquén,Santa teresa,1.0,Convenience Store,Restaurant,Farmers Market,Bakery,Athletics & Sports,Fish & Chips Shop,Café,Fast Food Restaurant,Pizza Place,Cultural Center
44,Usaquén,Torcoroma,1.0,Convenience Store,Restaurant,Bookstore,Bakery,Fast Food Restaurant,Café,Farmers Market,Pizza Place,Cuban Restaurant,Dumpling Restaurant
49,Usaquén,Darandelos,1.0,Restaurant,Italian Restaurant,Café,Burger Joint,Bar,Coffee Shop,South American Restaurant,Tapas Restaurant,BBQ Joint,Asian Restaurant
51,Usaquén,Guanoa,1.0,Restaurant,Italian Restaurant,Café,Burger Joint,Bar,Coffee Shop,South American Restaurant,Tapas Restaurant,BBQ Joint,Asian Restaurant


#### Examining cluster 3

In [123]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 2, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
135,Chapinero,San luis altos del cabo,2.0,Mountain,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Fabric Shop,Donut Shop


#### Examining cluster 4

In [124]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 3, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Usaquén,La floresta de la sabana,3.0,South American Restaurant,Pizza Place,Deli / Bodega,Bakery,Latin American Restaurant,Farmers Market,Event Space,Food Truck,Supermarket,Steakhouse
8,Usaquén,El verbenal,3.0,Gym,Seafood Restaurant,Ice Cream Shop,Pizza Place,Bakery,Gastropub,Grocery Store,Yoga Studio,Drugstore,Dumpling Restaurant
18,Usaquén,San antonio norte,3.0,Bakery,Soccer Field,Music Venue,Food & Drink Shop,Seafood Restaurant,Factory,Falafel Restaurant,Fabric Shop,Farm,Drugstore
24,Usaquén,El redil,3.0,Seafood Restaurant,Food & Drink Shop,Soccer Field,Bakery,Event Service,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
25,Usaquén,La cita,3.0,Construction & Landscaping,Forest,Trail,Thrift / Vintage Store,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service
29,Usaquén,San juan bosco,3.0,Trail,Thrift / Vintage Store,Bakery,BBQ Joint,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service
30,Usaquén,Urbanización los laureles,3.0,Construction & Landscaping,Costume Shop,Fried Chicken Joint,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Exhibit
38,Usaquén,Milán,3.0,Convenience Store,Ice Cream Shop,Pizza Place,Soccer Field,Deli / Bodega,Grocery Store,Yoga Studio,Event Service,Dumpling Restaurant,Eastern European Restaurant
47,Usaquén,El toberín,3.0,Latin American Restaurant,Sporting Goods Shop,Pizza Place,Grocery Store,Bakery,Soccer Field,Kids Store,Burger Joint,Deli / Bodega,BBQ Joint
48,Usaquén,Babilonia,3.0,Latin American Restaurant,Sporting Goods Shop,Sandwich Place,Soccer Field,BBQ Joint,Bakery,Mobile Phone Shop,Fast Food Restaurant,Farmers Market,Accessories Store


#### Examining cluster 5

In [125]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 4, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,Usaquén,Francisco Miranda,4.0,Park,Yoga Studio,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space
252,Fontibón,Las brisas,4.0,Park,Music Store,Yoga Studio,Event Service,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space
299,Antonio Nariño,San Jorge Central,4.0,Park,Auto Garage,Soccer Field,Yoga Studio,Event Service,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space


#### Examining cluster 6

In [126]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 5, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Usaquén,Mirador del norte,5.0,Fried Chicken Joint,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Exhibit,Donut Shop
40,Usaquén,San cristóbal norte,5.0,Athletics & Sports,Fried Chicken Joint,Yoga Studio,Fish Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space
41,Usaquén,San cristóbal norte parte alta,5.0,Athletics & Sports,Fried Chicken Joint,Yoga Studio,Fish Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space
42,Usaquén,San cristóbal norte parte baja,5.0,Athletics & Sports,Fried Chicken Joint,Yoga Studio,Fish Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space
131,Chapinero,Toscana,5.0,Athletics & Sports,Event Service,BBQ Joint,Yoga Studio,Flea Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit
172,Teusaquillo,San luis,5.0,Smoke Shop,Athletics & Sports,Fish Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Yoga Studio


#### examining cluster 7

In [127]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 6, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
210,Fontibón,Santiago Batavia,6.0,Grocery Store,Yoga Studio,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space
223,Fontibón,Flandes,6.0,Grocery Store,Park,Yoga Studio,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service
240,Fontibón,Villa carmenza,6.0,Grocery Store,Latin American Restaurant,Food,Park,Fried Chicken Joint,Yoga Studio,Electronics Store,Drugstore,Dumpling Restaurant,Eastern European Restaurant


#### Examining cluster 8

In [128]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 7, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Usaquén,Chaparral,7.0,Lighthouse,Event Space,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Yoga Studio,Donut Shop


#### Explorting cluster 9

In [129]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 8, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Usaquén,Bosque de pinos,8.0,Pub,Event Space,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Exhibit,Dog Run


#### Exploring cluster 10

In [130]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 9, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Usaquén,Danubio,9.0,Construction & Landscaping,Event Space,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Exhibit,Fish Market
238,Fontibón,Versalles,9.0,Construction & Landscaping,Bar,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Fabric Shop


#### Explorting cluster 11

In [132]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 10, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
245,Fontibón,El triángulo,10.0,Theater,Yoga Studio,Event Space,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Exhibit


#### Exploring cluster 12

In [133]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 11, bogota_merged.columns[[0]+[2] + list(range(5, bogota_merged.shape[1]))]]

,Locality,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Usaquén,Canaima,11.0,Furniture / Home Store,Department Store,Burger Joint,Grocery Store,Tennis Stadium,Bus Station,BBQ Joint,Golf Course,Sandwich Place,Flower Shop
3,Usaquén,Balmoral norte,11.0,Clothing Store,Seafood Restaurant,Italian Restaurant,Café,Bakery,Hotel,Coffee Shop,Spa,Sushi Restaurant,Donut Shop
4,Usaquén,Buenavista,11.0,Fast Food Restaurant,Bus Station,Dog Run,Clothing Store,Lighthouse,Brewery,Yoga Studio,Event Service,Eastern European Restaurant,Electronics Store
7,Usaquén,El refugio de San Antonio,11.0,Pool,Fast Food Restaurant,Bakery,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service
9,Usaquén,Horizontes,11.0,Tennis Stadium,Tennis Court,Burger Joint,Café,Food Truck,Empanada Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
13,Usaquén,Marantá,11.0,Coffee Shop,Italian Restaurant,Clothing Store,Burger Joint,Café,Latin American Restaurant,Bookstore,Ice Cream Shop,BBQ Joint,Dessert Shop
14,Usaquén,Maturín,11.0,Supermarket,Food Court,Shopping Mall,BBQ Joint,Skate Park,Fast Food Restaurant,Fabric Shop,Exhibit,Event Space,Drugstore
15,Usaquén,Medellín,11.0,Theater,BBQ Joint,Sandwich Place,Bakery,Market,Fast Food Restaurant,Movie Theater,Brewery,Building,Burger Joint
19,Usaquén,Santandersito,11.0,Pool,Fast Food Restaurant,Bakery,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service
26,Usaquén,La granja norte,11.0,Bookstore,Athletics & Sports,Café,Mobile Phone Shop,Food Truck,Exhibit,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service
